In [1]:
import math
import numpy as np
import torch

from torch_harmonics.quadrature import legendre_gauss_weights, clenshaw_curtiss_weights
from torch_harmonics.legendre import legpoly, clm
from torch_harmonics import RealSHT, InverseRealSHT

import matplotlib.pyplot as plt

from plotting import plot_sphere

In [2]:
nlat = 32
nlon = 2*nlat
grid = "legendre-gauss"

# for quadrature and plotting
if grid == "legendre-gauss":
    lmax = mmax = nlat
    xq, wq = legendre_gauss_weights(nlat)
elif grid =="equiangular":
    lmax = mmax = nlat//2-1
    xq, wq = clenshaw_curtiss_weights(nlat)

sht = RealSHT(nlat, nlon, lmax=lmax, mmax=mmax, grid=grid)
isht = InverseRealSHT(nlat, nlon, lmax=lmax, mmax=mmax, grid=grid)

lat = np.arccos(xq)
omega = torch.pi * torch.from_numpy(wq).float() / nlat
omega = omega.reshape(-1, 1).repeat(1, nlon)

omega.sum()

In [3]:
# forming the Vandermonde matrix
nmodes = int(lmax * (lmax+1) / 2)
e = torch.zeros(nmodes, lmax, mmax, dtype=torch.complex64)

midx = lambda l,m : l*(l+1)//2 + m

for l in range(lmax):
    for m in range(l+1):
        e[midx(l,m), l, m] = 1.0

vdm = isht(e) # nmodes x nlat x nlon

let us visualize the some entries in the Vandermonde matrix. In principle, it should contain the Spherical Harmonics in ascending order:

In [4]:
plt_lmax = 6

fig = plt.figure(layout='constrained', figsize=(12, 8))
subfigs = fig.subfigures(plt_lmax, plt_lmax)

for l in range(plt_lmax):
    for m in range(l+1):
        plot_sphere(vdm[midx(l,m)], fig=subfigs[l, m], title=f"l={l}, m={m}", lat=(lat - np.pi/2))

now, let us compute the gramian matrix $S^T \mathop{\mathrm{diag}}(\omega) S$:

In [5]:
gramian = torch.einsum("iqr,jqr,qr->ij", vdm, vdm, omega)

In [6]:
# plt.pcolormesh(xlg, np.arange(0, nmodes), , cmap="plasma")
# plt.xlabel("x")
# plt.ylabel("l,m mode")
# plt.colorbar()

In [7]:
plt.imshow(gramian, interpolation="nearest", cmap="plasma")
plt.colorbar()

In [8]:
print(torch.max(gramian))
print(torch.argmax(gramian))
print(torch.min(gramian))
print(torch.argmin(gramian))

In [9]:
np.linalg.cond(gramian)

In [10]:
plt.plot(np.diag(gramian))

In [11]:
# forming the Vandermonde matrix
nmodes = int(lmax * (lmax+1) / 2)


e = torch.zeros(lmax, mmax, lmax, mmax, dtype=torch.complex64)
for l in range(lmax):
    for m in range(l+1):
        e[l, m, l, m] = 1.0

Sinv = isht(e).cfloat() # l x m x nlat x nlon

e = torch.zeros(nlat, nlon, nlat, nlon, dtype=torch.float32)
for lat in range(nlat):
    for lon in range(nlon):
        e[lat, lon, lat, lon] = 1.0

S = sht(e) # nlat x nlon x l x m

In [19]:
out = torch.einsum("lmop,lmqr->lopqr", Sinv, Sinv.conj())

In [50]:
plot_sphere(out[3, 29, 16].abs(), colorbar=True)

In [12]:
SS = torch.einsum("lmqr, oplm->qrop", Sinv, S)

In [13]:
plt.imshow(SS.reshape(nlat*nlon, nlat*nlon).abs(), interpolation="nearest", cmap="plasma")
plt.colorbar()


In [14]:
plt.imshow(SS[:,0,:,0].abs(), interpolation="nearest", cmap="plasma")
plt.colorbar()

In [15]:
field = torch.zeros(nlat, nlon)
field[16,3] = 1.0

plot_sphere(field, colorbar=True)
plot_sphere(isht(sht(field)), colorbar=True)